In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

## Simulate Image Sets with Blender for Fourier Ptychography
With these functions, it is possible to generate **Lo-Res image sets to be used with a Fourier Ptychography algorithm**. We suggest to see the [fpblend-examples notebook](../examples/fpblend-examples.ipynb).

[blendjupyter.ipynb](blendjupyter.ipynb) is a shared Jupyter module &#171;written to set up and manipulate cameras (`'bpy.types.Camera'`), scenes (`'bpy.context.scene'`)&#187;, among other things.

This notebook **does not use** a Python kernel, it [uses a Blender kernel](https://github.com/panzi/blender_ipython).

In [ ]:
import bpy
import math
import blendjupyter as blendj

In [ ]:
def gen_xyz_led_ring(leds, d, h):
    """Returns a list with the coordinates of each LED of a LED ring.
    
    Args:
        leds: A list with the number of LEDs per ring from the center
            to the edge of the illuminator.
        d: The distance in mm between neighboring LEDs.
        h: The distance in mm between the sample and the LED
            illuminator.
    """
    l_coords = list()
    if isinstance(leds, list):
        for led in leds:
            if isinstance(led, int) and led > 0:
                r = (d * led) / (2.0 * math.pi)
                theta_step = 360 // led
                range_theta = range(0, 360, theta_step)
                range_theta = range_theta[:led]
                for theta in range_theta:
                    x = r * math.cos(math.radians(theta))
                    y = r * math.sin(math.radians(theta))
                    z = h
                    l_coords.append((x, y, z))
    return l_coords

In [ ]:
def gen_xyz_led_grid(n, d, h):
    """Returns a list with the coordinates of each LED of a n-by-n LED
    grid.
    
    Args:
        n: Number of LEDs (int) per row or column in the LED grid.
        d: The distance in mm between neighboring LEDs.
        h: The distance in mm between the sample and the LED
            illuminator.
    """
    l_coords = list()
    x_max = math.floor((n / 2)) * d
    x_min = -x_max
    y_max = x_max
    y_min = x_min
    l_range = list(range(x_min, x_max + 1, d))
    for i in l_range:
        for j in l_range:
            l_coords.append((j, -i, h))  # (x, y, z) coordinate.
    return l_coords

In [ ]:
def create_led_illuminator(sc, l_coords, **kwargs):
    """Creates a LED grid in Blender with lamps type 'POINT' and
    returns them contained in a list.
    
    Args:
        sc: A Blender scene.
        l_coords: A list with (x, y, z) coordinates.
    """
    radiusled = kwargs.pop('radiusled', 1.25)  # Radius in mm of the
                                               # LED.
    coordcam = kwargs.pop('coorcam', (0, 0, l_coords[0][2] - 1))
    l_lamps = list()
    len_lcoords = len(l_coords)
    for i in range(len_lcoords):
        lamp_name = "lamp" + blendj.num_str_zeros(i,
                                                  len_lcoords,
                                                  matlab=True)
        lamp_data = bpy.data.lamps.new(name=lamp_name,
                                       type='POINT')
        lamp_data.energy = 0.0 # The lamp is off.
        lamp_object = bpy.data.objects.new(name=lamp_name,
                                           object_data=lamp_data)
        sc.objects.link(lamp_object)
        lamp_object.location = l_coords[i]
        lamp_object.select = True
        sc.objects.active = lamp_object
        bpy.ops.mesh.primitive_cylinder_add(location=l_coords[i],
                         radius=radiusled,
                         depth=0)  # Base model for the LED.
        l_lamps.append(lamp_object)
    return l_lamps

In [ ]:
def illuminate_step_by_step(cam, sc, l_lamps):
    """Turns on a lamp at time of the LED illuminator, takes a photo
    of the mesh and saves it.
    
    Args:
        cam: A Blender camera ('CAMERA').
        sc: A Blender scene.
        l_lamps: A list with Blender lamps ('LAMP').
    """
    len_llamps = len(l_lamps)
    path = blendj.find_out_dir(dirname='blend-lores-set-0001',
                               parentdir='microscope')
    for i in range(len_llamps):
        l_lamps[i].data.energy = 2.0
        blendj.shoot_cam(cam,
                         sc,
                         i,
                         len_llamps,
                         path,
                         prefix='lores-img_')
        l_lamps[i].data.energy = 0.0